In [16]:
import librosa
import numpy as np
import scipy
from sklearn.neighbors import NearestNeighbors
import pydub
import math
import os

In [42]:
#replace with the name of the input file
original_wav = "grandevalse2.wav"

In [18]:
os.listdir("songs")

['skrillex.wav',
 'moonlight4.wav',
 'fantaisieimpromptu1.wav',
 '.DS_Store',
 'moonlight1.wav',
 'fantaisieimpromptu2.wav',
 'moonlight2.wav',
 'moonlight3.wav',
 'fantaisieimpromptu3.wav',
 'grandevalse1.wav',
 'furelise1.wav',
 'prelude3.wav',
 'prelude2.wav',
 'metal1.wav',
 'furelise.wav',
 'grandevalse2.wav',
 'furelise2.wav',
 'metal3.wav',
 'prelude1.wav',
 'metal2.wav',
 'furelise3.wav',
 'grandevalse3.wav',
 'furelise4.wav',
 'metal4.wav',
 'rondoallaturca1.wav',
 'rondoallaturca2.wav']

In [43]:
songs_wav = [song for song in os.listdir("songs") if song[-3:] == "wav"]
if original_wav in songs_wav:
    songs_wav.remove(original_wav)

In [44]:
original, sample_rate = librosa.load("songs/" + original_wav)

In [45]:
song_data = [original] + [librosa.load("songs/" + song)[0] for song in songs_wav]

In [22]:
def remove_zeros(vec):
    temp = np.transpose(vec == 0)
    indices = np.argwhere(temp == False)
    return vec[indices[0][0]:indices[len(indices) - 1][0]]

In [23]:
songs = [remove_zeros(s) for s in song_data] 

In [24]:
fft_frame_size = 2000

In [25]:
def get_fft_chunks(time_data):
    num_samples= len(time_data)//fft_frame_size
    return [np.fft.fft(time_data[i*fft_frame_size:(i+1)*fft_frame_size]) for i in range(num_samples)]

In [26]:
# original_fft = get_fft_chunks(remove_zeros(songs[0]))

In [27]:
#testing the stackoverflow code
#2000 is the length of one window
freq = np.fft.fftfreq(fft_frame_size)
fft_freqs = [abs(freq[i]*sample_rate) for i in range(fft_frame_size)]

In [1]:
fft_freqs

NameError: name 'fft_freqs' is not defined

In [29]:
ranges = [40, 80, 120, 180, 300]

In [30]:
def get_magnitudes(fft):
    #return high_mags, a 2d array
    #high_mags[i][0] is for 0-40
    #high_mags[i][1] is for 40-80
    #etc. where i is in range(len(fft)), or each fft window 
    high_mags = [np.zeros(len(ranges)) for k in range(len(fft))]
    
    def max_mag_in_window(index, high_mags):
        nonlocal i
        #tuple in form (current highest magnitude, index of current highest magnitude)
        mag = (0, 0)
        while(fft_freqs[i] < ranges[index]): 
            curr_mag = math.log10(abs(window[i])+1)
            if curr_mag > mag[0]:
                mag = (curr_mag, i)
            i += 1
            high_mags[fft_window][index] = fft_freqs[mag[1]]
            
    for fft_window in range(len(fft)):
        window = fft[fft_window]
        i = 0
        #find the maximum magnitudes in each window of ranges (0-40, 40-80, etc.)
        for j in range(len(ranges)):
            max_mag_in_window(j, high_mags)
    return high_mags

In [31]:
#a key is a set of 5 magnitudes that are the greatest in the ranges 
#0-40, 40-80, 80-120, 120-180, and 180-300 Hz
#the values are an array of tuples of type (time, song name)
#where time is the "window" (an index) and song is a string such as "furelise"

def populate_database(mags, database, song_name):
    for i in range(len(mags)): #i = index of "window" so it corresponds to what we can consider "time" i suppose
        key = str(mags[i])
        if key not in database:
            database[key] = {}
        if song_name not in database[key]:
            database[key][song_name] = []
        database[key][song_name].append(i)
    

In [46]:
database = {}
#songs[0] is the original/input, we want to process it separately
for index, song in enumerate(songs[1:]):
    print("adding song ", songs_wav[index][:-4], "...")
    
    original_fft = get_fft_chunks(song)
    mags = get_magnitudes(original_fft)
    populate_database(mags, database, songs_wav[index][:-4])
    

adding song  schubertimpromptu1 ...
adding song  skrillex ...
adding song  schubertimpromptu2 ...
adding song  moonlight4 ...
adding song  schubertimpromptu3 ...
adding song  fantaisieimpromptu1 ...
adding song  moonlight1 ...
adding song  fantaisieimpromptu2 ...
adding song  moonlight2 ...
adding song  moonlight3 ...
adding song  fantaisieimpromptu3 ...
adding song  grandevalse1 ...
adding song  furelise1 ...
adding song  prelude3 ...
adding song  prelude2 ...
adding song  metal1 ...
adding song  furelise ...
adding song  furelise2 ...
adding song  metal3 ...
adding song  prelude1 ...
adding song  metal2 ...
adding song  furelise3 ...
adding song  grandevalse3 ...
adding song  furelise4 ...


In [47]:
#process original/input song 
original_fingerprint = {}
original_fft = get_fft_chunks(original)
mags = get_magnitudes(original_fft)
populate_database(mags, original_fingerprint, original_wav)

In [48]:
similarities = {key[:-4]:0 for key in songs_wav}

In [35]:
# print(database['[  33.075   44.1    110.25   121.275  231.525]'])

{'skrillex': [286, 322, 624, 811, 875, 1270, 1273, 1295, 1296, 1332, 1521, 1613], 'fantaisieimpromptu2': [1197], 'moonlight3': [747], 'furelise1': [0, 1673, 2356], 'metal1': [23, 28], 'furelise2': [959, 1511], 'furelise3': [1524], 'grandevalse3': [677], 'furelise4': [1299]}


In [49]:
#for each set of 5 notes in the original/input song, check if in other songs
            
for key in original_fingerprint.keys():
    if key in database:
        for song_name in database[key]:
            similarities[song_name] += len(database[key][song_name])

In [50]:
similarities

{'fantaisieimpromptu1': 1528,
 'fantaisieimpromptu2': 1131,
 'fantaisieimpromptu3': 219,
 'furelise': 415,
 'furelise1': 46,
 'furelise2': 21,
 'furelise3': 16,
 'furelise4': 546,
 'grandevalse1': 866,
 'grandevalse3': 453,
 'metal1': 101,
 'metal2': 722,
 'metal3': 287,
 'metal4': 0,
 'moonlight1': 955,
 'moonlight2': 1841,
 'moonlight3': 366,
 'moonlight4': 1046,
 'prelude1': 2165,
 'prelude2': 489,
 'prelude3': 2608,
 'rondoallaturca1': 0,
 'rondoallaturca2': 0,
 'schubertimpromptu1': 364,
 'schubertimpromptu2': 1152,
 'schubertimpromptu3': 1456,
 'skrillex': 1817}

In [51]:
def knn(k, sim_dict):
    sorted_dict = sorted(sim_dict, key=sim_dict.get, reverse=True)[:k]
    counts = {}
    for s in sorted_dict:
        name = s[:-1]
        if name not in counts:
            counts[name] = 0
        counts[name] += 1
    return max(counts, key=counts.get)

knn(3, similarities)

'prelude'

In [52]:
print(max(similarities, key=similarities.get))

prelude3


In [243]:
# similarities = {key[:-4]:0 for key in songs_wav}

In [244]:
# for key in original_fingerprint.keys():
#     original_indices = original_fingerprint[key][original_wav]
#     if key in database:
#         for song_name in database[key]:
#             database_indices = database[key][song_name]
#             differences = [original - database for original, database in zip(original_indices, database_indices)]
#             relative_differences = np.mean(np.array(differences))
#             sim = np.linalg.norm(np.array(differences - relative_differences))
#             if sim < 100:
#                 similarities[song_name] += 1
#             elif sim < 1000:
#                 similarities[song_name] += 0.5
#             else:
#                 similarities[song_name] += 0.1


In [217]:
# print(similarities)

In [218]:
# print(max(similarities, key=similarities.get))